In [1]:
import dataset
import glob
import matplotlib.pyplot as plt
import os
import time

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 경고 메시지 제거
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 사용할 모델 선택

In [3]:
import models.resnet
import models.resnet101
import models.efficientnet
import models.efficientnetb5
import models.efficientnetb7

model_arr = [models.resnet, models.resnet101, models.efficientnet, models.efficientnetb5, models.efficientnetb7]

# 데이터 불러오기

In [6]:
#train_sql = test_sql = "SELECT FileLocation, Gender, Age, Dialect FROM json \
#        WHERE Gender != 'NotProvided' and Age != 'NotProvided' and Dialect != 'NotProvided' \
#        and FileLocation not like '%zzmt%' \
#        and cast(FileLength as real) > 6 and cast(FileLength as real) < 8 \
#        ORDER BY random()"
#train_ds, val_ds, test_ds = dataset.load_database(train_sql, test_sql)
_, _2, test_ds = dataset.load_dataset("voice_dialect_ds:1.0.0")

In [7]:
test_ds = models.resnet.preprocess_dataset(test_ds)
test_ds = test_ds.batch(8)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
for audio, labels in test_ds.take(1):
    print(audio.shape)

(8, 498, 32, 3)


# 모델 불러오기

In [9]:
def get_model(model, input_layer):
    model_layer = model.get_model()
    model_layer.compile()
    latest_checkpoint_path = os.path.join("checkpoint", model.model_name, "latest.ckpt")
    model_layer.load_weights(latest_checkpoint_path)

    return model_layer(input_layer)

In [10]:
output_names = ["age", "dialect", "gender"]
input_layer = tf.keras.Input(shape=(498, 32, 3))
layer_arr = [get_model(model, input_layer) for model in model_arr]

#input_reduce_layer = tf.keras.layers.Lambda(lambda inp: inp[:, :, :, 0], output_shape=(498, 32))(input_layer)
#lstm_layer_arr = [get_model(models.dvector_single, input_reduce_layer)]
#layer_arr.extend(lstm_layer_arr)

output_layer = [tf.keras.layers.Average(name=output_names[i])([output[i] for output in layer_arr]) for i in range(len(output_names))]

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
#model.compile()

model.compile(
          metrics={
              "age": ['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(2), tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(len(dataset.age_list))],
              "dialect": ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(len(dataset.dialect_list))],
              "gender": ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(1, threshold=0.5)],
          })

In [11]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 498, 32, 3)] 0                                            
__________________________________________________________________________________________________
model (Functional)              [(None, 6), (None, 6 26747789    input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Functional)            [(None, 6), (None, 6 45786637    input_1[0][0]                    
__________________________________________________________________________________________________
model_2 (Functional)            [(None, 6), (None, 6 20440684    input_1[0][0]                    
____________________________________________________________________________________________

In [12]:
model.evaluate(test_ds)

1560/1560 [==============================] - 1884s 1s/step - loss: 0.0000e+00 - age_accuracy: 0.4453 - age_top_k_categorical_accuracy: 0.7072 - age_auc: 0.8141 - age_precision: 0.5956 - age_recall: 0.2278 - age_f1_score: 0.4295 - dialect_accuracy: 0.8289 - dialect_auc_1: 0.9686 - dialect_precision_1: 0.8815 - dialect_recall_1: 0.7872 - dialect_f1_score: 0.8084 - gender_accuracy: 0.9586 - gender_auc_2: 0.9787 - gender_precision_2: 0.9584 - gender_recall_2: 0.9660 - gender_f1_score: 0.9622


[0.0,
 0.44526365399360657,
 0.7072447538375854,
 0.814051628112793,
 0.5956421494483948,
 0.22784100472927094,
 array([0.44612354, 0.45926875, 0.3476773 , 0.3511905 , 0.34829932,
        0.62452364], dtype=float32),
 0.8288988471031189,
 0.9686100482940674,
 0.8815400004386902,
 0.7872255444526672,
 array([0.98811847, 0.71799564, 0.87288356, 0.8299235 , 0.6089881 ,
        0.83266073], dtype=float32),
 0.9586471915245056,
 0.9786829352378845,
 0.9583698511123657,
 0.9659894108772278,
 array([0.9621645], dtype=float32)]

In [ ]:
model.save_weights("ensemble_model.ckpt")